In [5]:
import pandas as pd
import os
import json

from pprint import pprint

import chart_studio.plotly as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
import random

init_notebook_mode(connected=True)

# Multi-Objective Optimisation of Spiking Neural Networks
## Abstract
Spiking neural networks (SNNs) communicate through the all-or-none spiking activity of neurons. However, fitting the large number of SNN model parameters to observed neural activity patterns, e.g. in biological experiments, remains a challenge. Previous work using genetic algorithm (GA) optimisation on a specific efficient SNN model, using the Izhikevich neuronal model, was limited to a single parameter and objective. This work applied a version of GA, called non-dominated sorting GA (NSGA-III), to demonstrate the feasibility of performing multi-objective optimisation on the same SNN. We focus on searching for network connectivity parameters to achieve target firing rates of excitatory and inhibitory neuronal types, including across different network connectivity sparsity. We showed that NSGA-III could readily optimise for various firing rates. Notably, when the excitatory neural firing rates were higher than or equal to that of inhibitory neurons, the errors were small. Moreover, when connectivity sparsity was considered as a parameter to be optimised, the optimal solutions required sparse network connectivity. We also found that for excitatory neural firing rates lower than that of inhibitory neurons, the errors were generally larger. Overall, we have successfully demonstrated the feasibility of implementing multi-objective GA optimisation on network parameters of recurrent and sparse SNN.

## Pareto front convergence

In [26]:
def plot_animated_frontier_convergence(frontiers):
    # Ensure frontiers are sorted by generation
    frontiers = frontiers.sort_values(['generation'], ascending=True)
    
    frames = []
    for generation, solutions in frontiers.groupby('generation'):
        frames.append( go.Frame( data=[
            go.Scatter(
                x=solutions['ex_firing_error'],
                y=solutions['inhib_firing_error'],
                mode='markers'
            )
        ]))
        
    x_max = max(frontiers['ex_firing_error'])+.1
    x_min = min(frontiers['ex_firing_error'])-.1
    
    y_max = max(frontiers['inhib_firing_error'])+.1
    y_min = min(frontiers['inhib_firing_error'])-.1
        
    fig = go.Figure(
        data = [
            go.Scatter(x=[], y=[])
        ],
        layout=go.Layout(
            height=700,
            width =700,
            xaxis=dict(range=[x_min, x_max], autorange=False, zeroline=False),
            yaxis=dict(range=[y_min, y_max], autorange=False, zeroline=False),
            title_text="Pareto frontier over time", 
            xaxis_title='Excitatory firing error',
            yaxis_title='Inhibiatory firing error',
            hovermode="closest",
            updatemenus=[
                dict(
                    type="buttons",
                    buttons=[
                        dict(
                            label="Play", method="animate", 
                            args=[
                                None, 
                                dict(
                                    frame=dict(duration=50, redraw=False),                           
                                    transition=dict(duration=10),
                                    fromcurrent=True,
                                    mode='immediate'
                                )
                            ]
                        )
                    ]
                )
            ],
        ),
        frames = frames
    )
    
    fig.show()

In [27]:
def plot_all_generations(frontiers):
    frontiers = frontiers.sort_values(['generation'], ascending=True)
              
    x_max = max(frontiers['ex_firing_error'])+.1
    x_min = min(frontiers['ex_firing_error'])-.1
    
    y_max = max(frontiers['inhib_firing_error'])+.1
    y_min = min(frontiers['inhib_firing_error'])-.1
    
    fig = go.Figure(
        data = [
            go.Scatter(
                x=frontiers['ex_firing_error'], 
                y=frontiers['inhib_firing_error'],
                mode='markers',
                marker=dict(
                    color=frontiers['generation'],
                    showscale=True,
                ),
            )
        ],
        layout=go.Layout(
            height=700,
            width =700,
            xaxis=dict(range=[x_min, x_max], autorange=False),
            yaxis=dict(range=[y_min, y_max], autorange=False),
            xaxis_title='Excitatory error',
            yaxis_title='Inhibiatory error',
            coloraxis_colorbar=dict(
                title="Generation",
            ),
            title_text="Pareto frontier", 
            hovermode="closest",

        )
    )
    
    fig.show()

In [28]:
frontiers = pd.read_csv('/Users/fitzgerald/thesis/src/results/frontier/20210414-101710/frontier.csv')
plot_animated_frontier_convergence(frontiers)
plot_all_generations(frontiers)

## Firing rate optimisation
There are two parts to this study. First, for any fixed network connectivity fraction $ f $, we search for $ge$ and $gi$ parameters that minimize the (absolute) difference (i.e. the error) between the population and time-averaged firing rate of the excitatory neurons and inhibitory neurons and their respective targeted values. Second, we include the parameter $f$ together with $ge$ and $gi$ to form a set of parameters to be optimized with respect to the population- and time-averaged firing rates of the excitatory and inhibitory neurons.

In [9]:
from snnmoo.snn import SNN

model = SNN()
firings = model.run_network()
firings.plot_firings()

## Methods
 * Jobs were run on the kelvin2 cluster
 * Managed jobs using slurm
 * Results were downloaded and analysised locally

In [10]:
from typing import Dict, List, Tuple, Any
import numpy as np
import pandas as pd

from pymoo.algorithms.nsga2 import NSGA2
from pymoo.model.problem import Problem
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter
import os
import json

import sys
import datetime

from snnmoo.snn import SNN, SNNFirings

def snn_firing_test(
    ex_target_arg: float,
    in_target_arg: float,
    sparsity: float,
    max_error: float,
    generations: int=100,
    pop_size: int=100
    ) -> Tuple[Any, Dict]:
    ''' Create a new problem class and run to get results '''

    class SNNProblem(Problem):

        def __init__(self) -> None:
            super().__init__(n_var=2,
                             n_obj=2,
                             n_constr=2,
                             xl=np.array([0, 0]),
                             xu=np.array([2, 2]),
                             elementwise_evaluation=True)

        def _evaluate(self, chromosome: np.array, out: Dict, *args: List, **kwargs:  Dict) -> None:
            model = SNN(
                ge = chromosome[0],
                gi = chromosome[1],
                sparsity = sparsity,
            )

            firing = model.run_network()

            ex_target = ex_target_arg
            ex_distance = abs(ex_target - firing.score()['ex_firing'])

            in_target = in_target_arg
            in_distance = abs(in_target - firing.score()['in_firing'])

            # objective
            out["F"] = [ex_distance, in_distance]
            # constraint
            out["G"] = [ex_distance-max_error, in_distance-max_error]


    problem = SNNProblem()

    algorithm = NSGA2(pop_size=pop_size)

    res = minimize(problem,
                   algorithm,
                   ("n_gen", generations),
                   verbose=True,
                   save_history=True,
                   seed=1)

    params = {
        'ex_target':   ex_target_arg,
        'in_target':   in_target_arg,
        'sparsity' :   sparsity,
        'max_error':   max_error,
        'generations': generations,
        'pop_size':    pop_size,
    }

    return res, params

def serialse_results(res: Any, params: Dict, output: str) -> None:
    os.mkdir(output)
    n_evals = np.array([e.evaluator.n_eval for e in res.history])
    opt = np.array([e.opt[0].F for e in res.history])

    x=np.vstack([n_evals,opt[:,0],opt[:,1]]).T

    d = pd.DataFrame(x, columns=['n_evals', 'ex', 'inhib'])
    d.to_csv(f'{output}/opt.csv')

    frontier = pd.DataFrame(res.F, columns=['ex_firing_error','inhib_firing_error'])
    frontier.to_csv(f'{output}/frontier.csv')

    with open(f'{output}/params.json', 'w') as f:
        json.dump( params, f )

def main():
    firing_rates = [
        (2,10),
        (10,2),
        (2,2)
    ]

    sparsities = [
        1, .5, .2
    ]

    max_error = 5

    for sparsity in sparsities:
        for ex, inh in firing_rates:
            res, params = snn_firing_test(ex, inh, sparsity, max_error, pop_size=50, generations=25)
            time_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            serialse_results(res, params, f'/users/jfitzgerald/results/constraint/{time_str}')


In [11]:
def plot_3x3_grid(run_dir_path):
    '''Gather files from combined firing optimisation'''
    files = list(map(lambda x: f'{run_dir_path}/{x}', os.listdir(run_dir_path)))
    
    file_firing = []
    for filename in files:
        params = json.load(open(f'{filename}/params.json'))
        
        file_firing.append((
            filename,
            params,
            pd.read_csv(f'{filename}/frontier.csv'),
            pd.read_csv(f'{filename}/opt.csv')
        ))
      
    # Sort results by sparsity, excitatory_firing, inhibitory firing
    firings = sorted(
        file_firing, 
        key=lambda x: (-x[1]['sparsity'],-x[1]['ex_target'],x[1]['in_target'])
    )

    plot_titles = [
        f'f: {params["sparsity"]}, exc: {params["ex_target"]}, inh: {params["in_target"]}'
        for filename, params, frontier, opt in firings
    ]

    fig = make_subplots(
        rows=3, cols=3,
        subplot_titles=plot_titles,
        x_title='Excitatory neural firing rate error (Hz)',
        y_title='Inhibitory neural firing rate error (Hz)',
    )

    positions =[
        (1,1),(1,2),(1,3),
        (2,1),(2,2),(2,3),
        (3,1),(3,2),(3,3) 
    ]

    for i, (filename, params, frontier, opt) in enumerate(firings, start=1):
        position = positions[i-1]

        fig.add_trace(go.Scatter(
                x=frontier['inhib_firing_error'],
                y=frontier['ex_firing_error'],
                name='excitatory connections',
                mode='markers',
                marker={
                    'color':'black'
                },
            ), 
            row=position[0], col=position[1]
        )

    fig.update_layout(
        height=900, width=900, title='Final frontiers of firing error',

    )


    fig.show()

## Constrained search
Attempted to reduce errors by reducing the set of solutions that are accepted into the frontier. With a fixed max error of 5 the search has much hight errors for all combanations, and finds no possible solutions with a firing rate of 10,2.

In [12]:
plot_3x3_grid('/Users/fitzgerald/thesis/src/results/constraint')

## Combined firing optimisation
Attempted to reduce errors by minimising the sum of the errors
$ r_3 = r_1 + r_2 $

In [13]:
plot_3x3_grid('/Users/fitzgerald/thesis/src/results/combined')

## Sum squared errors
An attempt to minimse the distance from the origin $ r_3 = \sqrt{r_1^2 + r_2^2} $

In [14]:
plot_3x3_grid('/Users/fitzgerald/thesis/src/results/squared_error')

# Conclusions
 * Genetic algorithms are a brute force way to solve optimisation problems
 * Feature engineering can be helpful to minimise errors
 
In this work, we have successfully applied a version of the GA algorithm, called NSGA-III, in search for optimal model parameter of a cortical column-like recurrent SNN consisting of coupled populations of excitatory and inhibitory neurons. In particular, we were able to search for optimal connectivity parameters with respect to two network firing rate outputs in parallel. The connectivity parameters used were the range of the synaptic weights (ge and gi), and later the level of connectivity sparsity (f, fraction of network connections). We defined the NSGA-III objective function as the distance, i.e. (absolute) difference, between the simulated and targeted population- and time-averaged firing rates of the excitatory and inhibitory neurons.

### Further work
In this work, we have performed GA-based parameter optimisation for only connectivity parameters. With the convenience of the NSGA-III algorithm, it should be relatively easy to incorporate other model parameters e.g. intrinsic neuronal model parameters, (thalamic) input current and neuronal noise level. A limitation of our work was the relatively large errors for certain combinations of target firing rates. Future work may require modification of the algorithm, such as additional constraints. Future work could also employ similar methods to search for model parameters to fit high-dimensional firing-rate data which rich in temporal structure, and compare performances with other methods, including method using backpropagation through time, and explore how such algorithms can be applied to recurrent SNNs in neuromorphic computing system.

### Confrence paper accepted for ISSC 2021